## Task 1 : Less than k

Given a positive integer “k” and a list of integer numbers, look for the numbers within the
list, that are less than k. Consider an appropriate number of qubits and explain why your
proposal is valid for all kinds of numbers in case

## Proposed Solution :

To find the integers in a list l that are less than a positive integer k, the first thing that we do is to eliminate an obvious obstacle, which is the negative integers. After that, we can compare the integer k with each element in l by subtracting k with each element in l. We use Quantum Fourier Transform (QFT) for the subtraction. 

To do the subtraction, we can encode a number in binary format, and we can represent a binary number as a quantum state by using the computational basis as the following
$$
\ket{m} = \ket{q_0q_1...q_{n-1}}
$$
where the formula to obtain the decimal number is
$$
m = \sum_{i=0}^{n-1}2^{n-1-i}q_i.
$$
We can also represent a number by using another basis. In this case, we use a Fourier basis. In Fourier basis, all the states will be represented via qubits in the XY-plane of the Bloch sphere. This is how we can represent a number in this basis. Suppose we want to represent the number m by using n qubits, we can set the phase for the j-th qubit to be 
$$
\alpha_j = \frac{2m\pi}{2^j}.
$$


In [1]:
import pennylane as qml
import numpy as np

# A helper function that returns the bit length of the input integer
def bitlen(num):
    binary_string = bin(num)[2:]  
    binary_array = [int(bit) for bit in binary_string]
    return len(binary_array)

# 
def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])
        


Here's how we can use QFT to do the subtraction:
1. We use QFT to convert $\ket{m}$ from computational basis into Fourier basis
2. We use the Rz gate to rotate the j-th qubit by the angle $-\frac{2p\pi}{2^j}$, which leads to the phases $\frac{2(m-p)\pi}{2^j}$
3. We convert back the number into the computational basis to obtain $m-p$

We set the number of qubits to be n + 1, where n is the bit length of k. By doing this, the result in computational basis of subtraction of k with a positive integer that is less than k will have 0 as the first qubit, while the subtraction of k with the positive integer that is larger than k will have 1 as the first qubit. 

In [2]:
def execute(m,k):
    n_wires = bitlen(k) + 1
    dev = qml.device("default.qubit", wires=n_wires, shots=1)
    @qml.qnode(dev)
    def subtract(m, k):
        qml.BasisEmbedding(m, wires=range(n_wires))  # m encoding
        qml.QFT(wires=range(n_wires))  # step 1   
        add_k_fourier(k, range(n_wires))  # step 2   
        qml.adjoint(qml.QFT)(wires=range(n_wires))  # step 3   
        return qml.sample()
    return subtract(m,k)

Before we run this procedure, we have to eliminate one more obvious obstacle, that is the set of positive integers that the bit length are larger than the bit length of k. Because they are obviously larger than k. 

In [16]:
def less_than_k(k,l):
    less = []
    print("Binary subtraction result:")
    for i in l:
        if i < 0:
            less.append(i)
        elif execute(-i,k)[0] == 0 and k != i and bitlen(i) <= bitlen(k):
            print("integer: ", i, "Binary subtraction:", execute(-i,k))
            less.append(i)
    return less

In [17]:
le = less_than_k(5,[-20,-10,-4,1,2,3,4,5,6,7,8,100,1000,10000])
print("Less than k:", le)

Binary subtraction result:
integer:  1 Binary subtraction: [0 1 0 0]
integer:  2 Binary subtraction: [0 0 1 1]
integer:  3 Binary subtraction: [0 0 1 0]
integer:  4 Binary subtraction: [0 0 0 1]
Less than k: [-20, -10, -4, 1, 2, 3, 4]


## Discussion

Our proposed solution for this task is by using subtraction to compare an integer k with each element in a list l. This subtraction was done by using the Quantum Fourier Transform. This procedure requires n+1 qubit where n is the bit length of the binary representation of k. However, for this proposal to work properly, we must eliminate two obvious obstacles; the negative elements in the list and the numbers whose bit length of the binary representation is larger than the bit length of the binary representation of k. By doing this, our proposal works for every positive integer k and for every list of integers l.

## Reference

1. Basic Arithmetic with the Quantum Fourier Transform (QFT)." PennyLane Demos, PennyLane, https://pennylane.ai/qml/demos/tutorial_qft_arithmetics/.